In [24]:
import glob
import datetime as dt
import pandas as pd

df = pd.DataFrame()
path = r'/media/data/coding/strom/stromnetzgraz/raw_import/'
all_files = glob.glob(path + "/*.csv")

df_raw = pd.concat((pd.read_csv(r,
                                decimal=',',
                                parse_dates=['Ablesezeitpunkt'],
                                converters={'Ablesezeitpunkt': lambda t: dt.datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.%f%z').date()},
                                usecols=lambda x: x in ['Ablesezeitpunkt', 'Zaehlerstand Einheitstarif', 'Verbrauch Einheitstarif'],
                                )
                    for r in all_files))


In [25]:
df_raw

,Ablesezeitpunkt,Zaehlerstand Einheitstarif,Verbrauch Einheitstarif
0,2023-01-01,2123.111,6.419
1,2023-01-02,2129.401,6.29
2,2023-01-03,2135.028,5.627
3,2023-01-04,2139.98,4.952
4,2023-01-05,2144.119,4.139
...,...,...,...
85,2023-03-27,2580.975,9.433
86,2023-03-28,2583.444,2.469
87,2023-03-29,2586.428,2.984
88,2023-03-30,2591.248,4.82
